# Stochastic Compute Metadata Summary

This is a proof of concept summary notebook, created using data from this stac [catalog](https://ffrd.cloud.dewberryanalytics.com/projects/kanawha/stac), to provide high level overview of the results from a stochastic compute.

---

# --Notebook Name Here--
## Storms Summary

The map below shows the storm centers for all events simulated. Smaller events generating < 1.0" of excess precipitation are shown in blue, and larger events are shown in red. Circles are also sized based on excess precipitation.

Hover over the points to get the date, and click on the point to see the execess precipitation for that event. Go to [stormviewer](https://storms.dewberryanalytics.com/) to get addtional information.

In [10]:
from pystac_client import Client
import pystac
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import folium

client = Client.open("https://uampjfpbwi.us-east-1.awsapprunner.com/")

model_ids = [
    "BluestoneLocal",
    "BluestoneUpper",
    "Coal",
    "ElkMiddle",
    "ElkSutton",
    "GauleyLower",
    "GauleySummersville",
    "Greenbrier",
    "Little",
    "LowerKanawha",
    "LowerNew",
    "MiddleNew",
    "UpperKanawha",
    "UpperNew",
]

desired_model_id_index = 8

In [11]:
def get_props(collection, model_id):
    rows = []
    for i in range(1,1001):
        item_id = f"{model_id}-{i:04}"
        try:
            item = collection.get_item(item_id)
            properties = item.properties

            row = {
                'computation_time_total': properties.get("results_summary:computation_time_total"),
                'error_percent': properties.get("volume_accounting:error_percent"),
                'precipitation_excess_inches': properties.get("volume_accounting:precipitation_excess_inches"),
                'Historic_Storm_Date': properties.get("FFRD:storm_historic_date"),
                'longitude': properties.get("FFRD:longitude"),
                'latitude': properties.get("FFRD:latitude"),
            }
            rows.append(row)
        except:
            print(f"{item_id} not available.")

    df = pd.DataFrame(rows)
    df['geometry'] = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.crs = "EPSG:4326"

    return gdf

In [12]:
client
collection = client.get_collection("kanawha-simulations-march-2024")

In [13]:
model_id = model_ids[desired_model_id_index]
gdf = get_props(collection, model_id)

In [ ]:
geom_collection = client.get_collection("kanawha-models-march-2024")
geom_item = client.get_item(model_id)

In [ ]:

m = folium.Map(location=[gdf['latitude'].mean(), gdf['longitude'].mean()], zoom_start=7, tiles='OpenStreetMap',width='80%')

item = collection.get_item(f"{model_id}-0001")
folium.GeoJson(item.geometry, style_function=lambda x: {'fillColor': 'black', 'color': 'black'}).add_to(m)

for idx, row in gdf.iterrows():
    color = 'red' if row['precipitation_excess_inches'] > 1 else 'blue'  # Set the color to green if 'precip' is greater than 0.75, otherwise set it to black
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=row['precipitation_excess_inches'] + 2,  # Set the radius to the 'precip' property
                        popup=row['precipitation_excess_inches'],
                        tooltip=row['Historic_Storm_Date'],
                        fill=True,
                        fill_color=color,  # Set the fill color to blue
                        color=color  # Use the color variable to set the border color
                        ).add_to(m)


m



# Summary Statistics 

---

## Total Compute Time

In [ ]:
# Convert time column to total minutes (divide total seconds by 60)
gdf['computation_time_min'] = pd.to_timedelta(gdf['computation_time_total']).dt.total_seconds() / 60

plt.figure(figsize=(10, 6))
plt.hist(gdf['computation_time_min'], bins=20, color='red', edgecolor='black')
ymin, ymax = plt.ylim()

plt.ylim(ymin, np.ceil(ymax))

plt.yticks(np.arange(ymin, np.ceil(ymax)+1, 100))
plt.grid(False)

plt.xlabel('Total Computation Time (min)')
plt.ylabel('Frequency')
plt.title('Total Computation Time')
plt.show()

Note that the compute time may vary based on CPU used in cloud compute.

## Volume Error Summary

In [ ]:
plt.figure(figsize=(10, 6))
gdf["error_percent"].plot(kind='hist', bins=50, color='blue', edgecolor='black')
plt.xlabel('Error %')
plt.ylabel('Frequency')
plt.title(f'Percent Error')

ymin, ymax = plt.ylim()

plt.ylim(ymin, np.ceil(ymax))

plt.yticks(np.arange(ymin, np.ceil(ymax)+1, 50))
plt.grid(False)

plt.show()

## Excess Precipitation Summary

In [ ]:
plt.figure(figsize=(10, 6))
gdf["precipitation_excess_inches"].plot(kind='hist', bins=50, color='green', edgecolor='black')

plt.xlabel('Precipitation Excess (in.)')
plt.ylabel('Frequency')
plt.title(f'Precipitation Excess')

ymin, ymax = plt.ylim()

plt.ylim(ymin, np.ceil(ymax))

plt.yticks(np.arange(ymin, np.ceil(ymax)+1, 100))
plt.show()

# END